# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [9]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [10]:
# csv from WeatherPy output is saved as a dataframe.
cities_df = pd.read_csv("../WeatherPy/output_data/cities.csv")
cities_df.head()

,City,Latitude,Longitude,Maximum Temperature (°F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,Saint Paul Harbor,57.79,-152.41,59.000,58,1,7.20,US,2020-07-23
1,Rikitea,-23.12,-134.97,66.254,74,89,9.29,PF,2020-07-23
2,Cabo San Lucas,22.89,-109.91,80.996,75,5,0.45,MX,2020-07-23
3,Torbay,47.67,-52.73,59.000,87,75,2.10,CA,2020-07-23
4,Saint-Augustin,51.23,-58.65,56.588,75,12,3.08,CA,2020-07-23


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Base map is crafted
fig = gmaps.figure()

# Location and weights are defined for ease of reading
locations = cities_df[["Latitude", "Longitude"]].astype(float)
weights = cities_df["Humidity (%)"]

# Heat layer is crafted
heat_layer = gmaps.heatmap_layer(locations,
                                 weights=weights,
                                 max_intensity=100,
                                point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Show layer
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
# Locations are limited by personal preference.  No clouds, low wind, and temperatures between 70 and 80 degrees Farenheit.
location_df = cities_df.loc[cities_df["Cloudiness (%)"] == 0]\
                    .loc[cities_df["Wind Speed (mph)"] < 10]\
                    .loc[cities_df["Maximum Temperature (°F)"] < 80]\
                    .loc[cities_df["Maximum Temperature (°F)"] > 70]

# Drop rows with missing data
location_df.dropna(inplace=True)

# Output the first 10 locations fitting these criteria
location_df.head(10)

,City,Latitude,Longitude,Maximum Temperature (°F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
17,Carnarvon,-24.87,113.63,73.400,64,0,8.70,AU,2020-07-23
18,Kruisfontein,-34.00,24.73,72.266,23,0,0.63,ZA,2020-07-23
66,Port Alfred,-33.59,26.89,72.482,29,0,1.28,ZA,2020-07-23
79,Nouadhibou,20.93,-17.03,71.600,83,0,8.20,MR,2020-07-23
81,Suntar,62.14,117.63,77.360,48,0,2.93,RU,2020-07-23
103,Faanui,-16.48,-151.75,77.522,77,0,9.06,PF,2020-07-23
191,Verkhnyaya Inta,65.98,60.31,76.784,46,0,5.73,RU,2020-07-23
213,Boone,39.03,-92.33,73.004,90,0,2.61,US,2020-07-23
265,Karratha,-20.74,116.85,75.200,43,0,3.09,AU,2020-07-23
301,Shakawe,-18.37,21.85,70.340,23,0,3.86,BW,2020-07-23


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
# Dataframe for hotel information is crafted.
hotel_data = pd.DataFrame({"City":location_df["City"],
                         "Country":location_df["Country"],
                        "Lat": location_df["Latitude"],
                        "Lng": location_df["Longitude"]})

# Adding empty column for hotel name
hotel_data["Hotel Name"] = ""
hotel_data.head()

,City,Country,Lat,Lng,Hotel Name
17,Carnarvon,AU,-24.87,113.63,
18,Kruisfontein,ZA,-34.00,24.73,
66,Port Alfred,ZA,-33.59,26.89,
79,Nouadhibou,MR,20.93,-17.03,
81,Suntar,RU,62.14,117.63,


In [14]:
# Hotel names are grabbed from google places and placed in hotel name column
# Base url is defined
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Set parameters that will not change each iteration
params = {"type": "lodging",
          "radius": 5000,
          "key": g_key}
         
# iterrows iterates through rows of hotel dataframe
for index, row in hotel_data.iterrows():
    
    # Add location parameter
    params["location"] = f"{row['Lat']},{row['Lng']}"
    
    # Make api calls
    response = requests.get(base_url, params=params).json()
    
    try:
        hotel_data.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except:
        0

        
# Drops every location which does not have a hotel name, whether no value or an empty string. 
hotel_data.dropna(inplace=True)
hotel_df = hotel_data.loc[hotel_data["Hotel Name"] != ""]

# Output the remaining hotels
hotel_df.head(10)

,City,Country,Lat,Lng,Hotel Name
17,Carnarvon,AU,-24.87,113.63,Hospitality Carnarvon
18,Kruisfontein,ZA,-34.00,24.73,Oyster Bay House Rental
66,Port Alfred,ZA,-33.59,26.89,The Halyards Hotel
79,Nouadhibou,MR,20.93,-17.03,El Medina
81,Suntar,RU,62.14,117.63,Gostevoy Dom
103,Faanui,PF,-16.48,-151.75,Bora-Bora Pearl Beach Resort & Spa
213,Boone,US,39.03,-92.33,Cottonwoods RV Park
265,Karratha,AU,-20.74,116.85,ibis Styles Karratha
301,Shakawe,BW,-18.37,21.85,Ditlhapi Guest House
320,Kyzyl-Suu,KG,42.34,78.00,Gostinitsa Nur-Ordo


In [15]:
# NOTE: Do not change any of the code in this cell
# Roger Roger

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [17]:
# Add marker layer ontop of heat map

# Grab all locations for hotels
locations = hotel_df[["Lat","Lng"]].astype(float)
text = []

for index, row in hotel_df.iterrows():
    text.append(f'Name: {row["Hotel Name"]} \nCity: {row["City"]} \nCountry: {row["Country"]}')

markers = gmaps.marker_layer(locations, hover_text=text)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))